<a href="https://colab.research.google.com/github/Maynex69/Gestion-de-ingresos-hospitalarios/blob/main/Limpieza%20y%20obtenci%C3%B3n%20de%20datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import altair as alt
!pip install altair_viewer
!pip install altair vega_datasets

# Carga el archivo CSV usando Polars
df = pd.read_csv('/content/dataset.csv')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 10.0 MB/s eta 0:00:00


In [2]:
# Reducimos la cantidad del "Billing Amount" para hacer los resultados más heterogéneos.

# Definir el porcentaje de filas a modificar y el nuevo valor
porcentaje_a_modificar = 35  # Porcentaje de filas a modificar
nuevo_valor = 10500  # Nuevo valor con el que reemplazar

# Calcular el número de filas a modificar
num_filas = len(df)
num_filas_a_modificar = int(num_filas * porcentaje_a_modificar / 100)

# Seleccionar aleatoriamente las filas a modificar
filas_a_modificar = np.random.choice(df.index, size=num_filas_a_modificar, replace=False)

# Reemplazar los valores en las filas seleccionadas
df.loc[filas_a_modificar, 'Billing Amount'] = nuevo_valor


In [3]:

#Transformación de datos
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])

# Elimina las columnas no necesarias
df_dropped = df.drop(columns=['Doctor', 'Hospital', 'Room Number', 'Name', 'Test Results' ])

#En este punto me di cuenta de que el Dataset tenía prácticamente la misma
#cantidad de pacientes por cada enfermedad, así que decidí borrar una cantidad
#al azar para que fuera un escenario más diverso.

porcentajes = {
    'Female': 0.12,
    'Male': 0.26
}

indices_a_eliminar = []

for valor, porcentaje in porcentajes.items():
  filas_objeto = df_dropped[df_dropped['Gender'] == valor]
  num_filas_a_eliminar = int(len(filas_objeto) * porcentaje)
  filas_a_eliminar_aleatorias = filas_objeto.sample(n=num_filas_a_eliminar, random_state=1)
  indices_a_eliminar.extend(filas_a_eliminar_aleatorias.index)

df_modificado = df_dropped.drop(index=indices_a_eliminar)

#Ahora tenemos un nuevo dataframe con datos más aleatorizados para el desarrollo del modelo
#Lamentablemente el resto de las columnas sigue teniendo valores muy similares



In [4]:
#Reducción del número de aseguradoras

porcentajes_insurance = {
    'Cigna': 0.32,
    'Medicare': 0.26,
    'Aetna' : 0.18,
    'UnitedHealthcare': 0.12,
    'Blue Cross': 0.4
}

indices_a_eliminar = []

for valor, porcentaje in porcentajes_insurance.items():
  filas_objeto1 = df_modificado[df_modificado['Insurance Provider'] == valor]
  num_filas_a_eliminar1 = int(len(filas_objeto1) * porcentaje)
  filas_a_eliminar_aleatorias1 = filas_objeto1.sample(n=num_filas_a_eliminar1, random_state=1)
  indices_a_eliminar.extend(filas_a_eliminar_aleatorias1.index)

  df_modificado1 = df_modificado.drop(index=indices_a_eliminar)


In [5]:
#Reducción del porcentaje de condiciones médicas

porcentajes_p = {
    'Diabetes':0.16,
    'Obesity':0.2,
    'Cancer':0.7,
    'Hypertension':0.3
}

indices_a_eliminar = []

for valor, porcentaje in porcentajes_p.items():
  filas_objeto2 = df_modificado1[df_modificado1['Medical Condition'] == valor]
  num_filas_a_eliminar2 = int(len(filas_objeto2) * porcentaje)
  filas_a_eliminar_aleatorias2 = filas_objeto2.sample(n=num_filas_a_eliminar2, random_state=1)
  indices_a_eliminar.extend(filas_a_eliminar_aleatorias2.index)

  df_modificado2 = df_modificado1.drop(index=indices_a_eliminar)



In [6]:
#Reducción del número de medicamentos

porcentajes_m = {
    'Penicillin':0.3,
    'Ibuprofen':0.4,
    'Aspirin':0.27,
    'Lipitor':0.2,
    'Paracetamol':0.8
}

indices_a_eliminar = []

for valor, porcentaje in porcentajes_m.items():
  filas_objeto3 = df_modificado2[df_modificado2['Medication'] == valor]
  num_filas_a_eliminar3 = int(len(filas_objeto3) * porcentaje)
  filas_a_eliminar_aleatorias3 = filas_objeto3.sample(n=num_filas_a_eliminar3, random_state=1)
  indices_a_eliminar.extend(filas_a_eliminar_aleatorias3.index)

  df_1 = df_modificado2.drop(index=indices_a_eliminar)

  #Creamos una variable que nos hable de cuántos días pasó hospitalizado el paciente

df_1['Date of Admission'] = pd.to_datetime(df_1['Date of Admission'])

df_1['Discharge Date'] = pd.to_datetime(df_1['Discharge Date'])

df_1['Hospitalization Days'] = (df_1['Discharge Date'] - df_1['Date of Admission']).dt.days

#Así ya tenemos preparado nuestro Dataframe



In [7]:
#Visualización de las condiciones más frecuentes en los últimos 5 años

# Contar las ocurrencias de cada valor en la columna
conteos = df_1['Medical Condition'].value_counts()

# Calcular el porcentaje de cada valor
porcentajes1 = (conteos / len(df_1)) * 100

# Crear un DataFrame con los porcentajes para Altair
df_porcentaje = porcentajes1.reset_index()
df_porcentaje.columns = ['Condition', 'Percentage']

# Total de filas
total_filas = len(df_1)

# Crear una gráfica de pastel con Altair
grafico = alt.Chart(df_porcentaje).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field='Percentage', type='quantitative', title='Percentage'),
    color=alt.Color(field='Condition', type='nominal', title='Condition'),
    tooltip=[alt.Tooltip(field='Condition', type='nominal', title='Condition'),
             alt.Tooltip(field='Percentage', type='quantitative', title='Percentage')]
).properties(
    title='Income causes',
    width=400,  # Ajusta el ancho del gráfico
    height=400  # Ajusta la altura del gráfico
)

# Crear un DataFrame para el texto total
df_texto_total = pd.DataFrame({
    'x': [0],
    'y': [0],
    'texto': [f'Number of patients: {total_filas}']
})

# Crear el gráfico de texto
texto_total = alt.Chart(df_texto_total).mark_text(
    align='center',
    baseline='middle',
    fontSize=14,
    dy=210  # Ajusta la posición vertical del texto
).encode(
    x=alt.X('x:O', axis=None),
    y=alt.Y('y:O', axis=None),
    text='texto:N'
).properties(
    width=400,  # Ancho para el gráfico de texto
    height=400  # Altura para el gráfico de texto
)

# Combinar el gráfico de pastel y el gráfico de texto
grafico_final = grafico + texto_total

# Mostrar el gráfico en Google Colab
grafico_final

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Asma: medicamentos más usados, costo total en promedio, diás de estancia en promedio...

In [8]:
#Los tres medicamentos más utilizados en Asthma#

# Filtrar las filas donde "Medical Conditions" es "Asthma"
df_asthma = df_1[df_1['Medical Condition'] == 'Asthma']

# Contar las ocurrencias de cada objeto en "Medicamentos"
conteos_medicamentos = df_asthma['Medication'].value_counts()

total_asthma  = len(df_asthma)

# Seleccionar los 3 medicamentos más frecuentes
top_3_medicamentos = conteos_medicamentos.head(3).reset_index()
top_3_medicamentos.columns = ['Medication', 'Count']

# Crear una gráfica de barras con Altair
grafico_barras = alt.Chart(top_3_medicamentos).mark_bar().encode(
    x=alt.X('Medication:O', title='Medication'),
    y=alt.Y('Count:Q', title='Number of uses', axis=alt.Axis(format='d')),
    color='Medication:N',
    tooltip=[alt.Tooltip(field='Medication', type='nominal', title='Medication'),
             alt.Tooltip(field='Count', type='quantitative', title='Number of uses')]
).properties(
    title='The three most used medications for "Asthma"',
    width=400,  # Ajusta el ancho del gráfico
    height=300  # Ajusta la altura del gráfico
)

# Crear un DataFrame para el texto total
df_texto_total = pd.DataFrame({
    'x': [0],
    'y': [0],
    'texto': [f'Patients with "Asthma": {total_asthma}']
})

# Crear el gráfico de texto
texto_total = alt.Chart(df_texto_total).mark_text(
    align='center',
    baseline='bottom',
    fontSize=14,
    dy=170  # Ajusta la posición vertical del texto
).encode(
    x=alt.X('x:O', axis=None),
    y=alt.Y('y:O', axis=None),
    text='texto:N'
).properties(
    width=400,  # Ancho para el gráfico de texto
    height=300  # Altura para el gráfico de texto
)

# Combinar el gráfico de barras y el gráfico de texto
grafico_final = grafico_barras + texto_total

grafico_final


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [9]:
#Calcular el gasto promedio por estancia hospitalaria

total_billing_amount = int(df_asthma['Billing Amount'].sum())

# Calculamos el promedio de los valores en la columna "Billing Insurance" para los registros con "Asthma"
average_billing_amount = int(df_asthma['Billing Amount'].mean())

#Calculamos el tiempo de estancia promedio de un paciente con Asma

average_hospitalization_days_asthma = df_asthma['Hospitalization Days'].mean()

# Crea la gráfica lineal con Altair

# Agrupa por número de días de hospitalización y calcula el costo promedio
average_cost_by_days_asthma = df_asthma.groupby('Hospitalization Days')['Billing Amount'].mean().reset_index()

# Definir el valor mínimo de costo para mostrar en la gráfica, por ejemplo, $500
min_cost = 17200

# Filtrar los datos para asegurarse de incluir solo los valores a partir de la cifra deseada
filtered_data = average_cost_by_days_asthma[average_cost_by_days_asthma['Billing Amount'] >= min_cost]

# Crear la gráfica lineal con Altair con un eje Y que comienza desde el costo mínimo especificado
chart = alt.Chart(filtered_data).mark_line(point=True).encode(
    x=alt.X('Hospitalization Days', title='Hospitalization Days'),
    y=alt.Y('Billing Amount', title='Average cost of hospitalization', scale=alt.Scale(domain=(min_cost, filtered_data['Billing Amount'].max()))),
    tooltip=['Hospitalization Days', 'Billing Amount']
).properties(
    title=f'Average cost of hospitalization for Asthma',
    width=600,
    height=400
)

print(f'Average days of hospitalization for patients with Asthma: {average_hospitalization_days_asthma:.2f} days')
print(f'Average cost of hospitalization for asthma: {average_billing_amount}')

chart


Average days of hospitalization for patients with Asthma: 15.58 days
Average cost of hospitalization for asthma: 20274


alt.Chart(...)

Diabetes: medicamentos más usados, costo total en promedio, diás de estancia en promedio...

In [10]:
#Los tres medicamentos más utilizados en Diabetes#

# Filtrar las filas donde "Medical Conditions" es "Diabetes"
df_diabetes = df_1[df_1['Medical Condition'] == 'Diabetes']

# Contar las ocurrencias de cada objeto en "Medicamentos"
conteos_medicamentos_diabetes = df_diabetes['Medication'].value_counts()

total_diabetes  = len(df_diabetes)

# Seleccionar los 3 medicamentos más frecuentes
top_3_medicamentos_diabetes = conteos_medicamentos_diabetes.head(3).reset_index()
top_3_medicamentos_diabetes.columns = ['Medication', 'Count']

# Crear una gráfica de barras con Altair
grafico_barras = alt.Chart(top_3_medicamentos_diabetes).mark_bar().encode(
    x=alt.X('Medication:O', title='Medication'),
    y=alt.Y('Count:Q', title='Number of uses', axis=alt.Axis(format='d')),
    color='Medication:N',
    tooltip=[alt.Tooltip(field='Medication', type='nominal', title='Medication'),
             alt.Tooltip(field='Count', type='quantitative', title='Number of uses')]
).properties(
    title='The three most used medications for "Diabetes"',
    width=400,  # Ajusta el ancho del gráfico
    height=300  # Ajusta la altura del gráfico
)

# Crear un DataFrame para el texto total
df_texto_total_diabetes = pd.DataFrame({
    'x': [0],
    'y': [0],
    'texto': [f'Patients with "Diabetes": {total_diabetes}']
})

# Crear el gráfico de texto
texto_total = alt.Chart(df_texto_total_diabetes).mark_text(
    align='center',
    baseline='bottom',
    fontSize=14,
    dy=170  # Ajusta la posición vertical del texto
).encode(
    x=alt.X('x:O', axis=None),
    y=alt.Y('y:O', axis=None),
    text='texto:N'
).properties(
    width=400,  # Ancho para el gráfico de texto
    height=300  # Altura para el gráfico de texto
)

#Calcular el gasto promedio por estancia hospitalaria

total_billing_amount_d = int(df_diabetes['Billing Amount'].sum())

# Calculamos el promedio de los valores en la columna "Billing Insurance" para los registros con "Asthma"
average_billing_amount_d = int(df_diabetes['Billing Amount'].mean())

#Calculamos el tiempo de estancia promedio de un paciente con Asma

average_hospitalization_days_d = df_diabetes['Hospitalization Days'].mean()

# Agrupa por número de días de hospitalización y calcula el costo promedio
average_cost_by_days_d = df_diabetes.groupby('Hospitalization Days')['Billing Amount'].mean().reset_index()

# Combinar el gráfico de barras y el gráfico de texto
grafico_final = grafico_barras + texto_total

print(f'Average days of hospitalization for patients with diabetes: {average_hospitalization_days_d:.2f} days')
print(f'Average cost of hospitalization for asthma: {average_billing_amount_d}')
grafico_final

Average days of hospitalization for patients with diabetes: 15.21 days
Average cost of hospitalization for asthma: 20334


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Artritis: medicamentos más usados, costo total en promedio, diás de estancia en promedio...

In [11]:
#Los tres medicamentos más utilizados en Artritis#

# Filtrar las filas donde "Medical Conditions" es "Arthritis"
df_arthritis = df_1[df_1['Medical Condition'] == 'Arthritis']

# Contar las ocurrencias de cada objeto en "Medicamentos"
conteos_medicamentos_arthritis = df_arthritis['Medication'].value_counts()

total_arthritis  = len(df_arthritis)

# Seleccionar los 3 medicamentos más frecuentes
top_3_medicamentos_arthritis = conteos_medicamentos_arthritis.head(3).reset_index()
top_3_medicamentos_arthritis.columns = ['Medication', 'Count']

# Crear una gráfica de barras con Altair
grafico_barras = alt.Chart(top_3_medicamentos_arthritis).mark_bar().encode(
    x=alt.X('Medication:O', title='Medication'),
    y=alt.Y('Count:Q', title='Number of uses', axis=alt.Axis(format='d')),
    color='Medication:N',
    tooltip=[alt.Tooltip(field='Medication', type='nominal', title='Medication'),
             alt.Tooltip(field='Count', type='quantitative', title='Number of uses')]
).properties(
    title='The three most used medications for "Arthritis"',
    width=400,  # Ajusta el ancho del gráfico
    height=300  # Ajusta la altura del gráfico
)

# Crear un DataFrame para el texto total
df_texto_total_arthritis = pd.DataFrame({
    'x': [0],
    'y': [0],
    'texto': [f'Patients with "Arthritis": {total_arthritis}']
})

# Crear el gráfico de texto
texto_total_arthritis = alt.Chart(df_texto_total_arthritis).mark_text(
    align='center',
    baseline='bottom',
    fontSize=14,
    dy=170  # Ajusta la posición vertical del texto
).encode(
    x=alt.X('x:O', axis=None),
    y=alt.Y('y:O', axis=None),
    text='texto:N'
).properties(
    width=400,  # Ancho para el gráfico de texto
    height=300  # Altura para el gráfico de texto
)

#Calcular el gasto promedio por estancia hospitalaria

total_billing_amount_a = int(df_arthritis['Billing Amount'].sum())

# Calculamos el promedio de los valores en la columna "Billing Amount" para los registros con "Arthritis"
average_billing_amount_a = int(df_arthritis['Billing Amount'].mean())

#Calculamos el tiempo de estancia promedio de un paciente con Artritis

average_hospitalization_days_a = df_arthritis['Hospitalization Days'].mean()

# Agrupa por número de días de hospitalización y calcula el costo promedio
average_cost_by_days_a = df_arthritis.groupby('Hospitalization Days')['Billing Amount'].mean().reset_index()

# Combinar el gráfico de barras y el gráfico de texto
grafico_final = grafico_barras + texto_total_arthritis

print(f'Average days of hospitalization for patients with arthritis: {average_hospitalization_days_a:.2f} days')
print(f'Average cost of hospitalization for arthritis: {average_billing_amount_a}')
grafico_final

Average days of hospitalization for patients with arthritis: 15.54 days
Average cost of hospitalization for arthritis: 20184


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Muy bien. Ya obtuvimos los datos respecto a las tres enfermedades más comunes dentro de nuestro Dataframe, así como los tres medicamentos más usados dentro de cada una de ellas, sus días promedio de estancia intrahospitalaria y el costo promedio  por hospitatilzación.

A continuación determinaremos el porcentaje que cada aseguradora tiene con base a los pacientes cuyos gastos médicos cubrieron.

In [12]:
# Contar el número total de pacientes
total_pacientes = len(df_1)

# Contar el número de pacientes por cada aseguradora
conteo_aseguradoras = df_1['Insurance Provider'].value_counts()

# Calcular el porcentaje de pacientes por cada aseguradora
porcentaje_aseguradoras = (conteo_aseguradoras / total_pacientes) * 100

# Convertir el resultado a un DataFrame para una presentación más clara
resultado = pd.DataFrame({
    'Insurance Provider': porcentaje_aseguradoras.index,
    'Percentage': porcentaje_aseguradoras.values
})

# Ordenar el resultado por porcentaje en orden descendente
resultado = resultado.sort_values(by='Percentage', ascending=False)

# Mostrar el resultado
print(resultado)

  Insurance Provider  Percentage
0   UnitedHealthcare   23.637173
1              Aetna   21.709815
2           Medicare   19.846066
3              Cigna   18.688379
4         Blue Cross   16.118568


Ahora veamos cuáles son las condicionas más manejadas por nuestra principal aseguradora.

In [13]:
# Filtra los datos para la aseguradora específica
aseguradora = 'UnitedHealthcare'
df_aseguradora = df_1[df_1['Insurance Provider'] == aseguradora]

# Contar el número de pacientes por condición médica para la aseguradora
conteo_condiciones = df_aseguradora['Medical Condition'].value_counts().reset_index()
conteo_condiciones.columns = ['Medical Condition', 'Count']

# Calcular el total de pacientes asegurados por la aseguradora
total_pacientes_aseguradora = len(df_aseguradora)

# Calcular el porcentaje para cada condición médica
conteo_condiciones['Percentage'] = (conteo_condiciones['Count'] / total_pacientes_aseguradora) * 100

# Crear el gráfico de pastel con Altair
chart = alt.Chart(conteo_condiciones).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field='Percentage', type='quantitative'),
    color=alt.Color(field='Medical Condition', type='nominal'),
    tooltip=['Medical Condition', 'Count', 'Percentage:Q']
).properties(
    title=f'Distribución de Condiciones Médicas Cubiertas por {aseguradora}'
)

# Crear un texto informativo con Altair
text = alt.Chart(pd.DataFrame({
    'text': [f'Total de pacientes asegurados: {total_pacientes_aseguradora}']
})).mark_text(
    align='center',
    baseline='middle',
    fontSize=12,
    dy=180
).encode(
    text='text:N'
)

# Combinar el gráfico y el texto en una sola visualización usando alt.layer
final_chart = alt.layer(chart, text).properties(
    height=350  # Ajusta la altura total de la visualización si es necesario
)

final_chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Ahora determinaremos cuánto suele costar en promedio cada enfermedad según su estancia intrahospitalaria.

In [14]:
# Agrupar los datos por la columna 'Medical Condition' y calcular el promedio del 'Billing Amount'
promedio_gasto = df_1.groupby('Medical Condition')['Billing Amount'].mean().reset_index()

promedio_gasto_ordenado = promedio_gasto.sort_values(by='Billing Amount', ascending=False)

# Renombrar las columnas para mayor claridad
promedio_gasto_ordenado.columns = ['Medical Condition', 'Average Billing Amount']

# Mostrar el DataFrame resultante
print(promedio_gasto_ordenado)

  Medical Condition  Average Billing Amount
5           Obesity            20660.572879
3          Diabetes            20334.824238
1            Asthma            20274.352595
4      Hypertension            20210.201751
0         Arthritis            20184.325647
2            Cancer            19635.949742


In [17]:
#Ahora solo por diversión dividiremos a las enfermedades más fracuentes según su edad.

# Agrupar por condición médica y calcular el promedio de edad
avg_age = df_1.groupby('Medical Condition')['Age'].mean().reset_index()

# Crear un gráfico de barras con Altair
chart = alt.Chart(avg_age).mark_bar().encode(
    x='Medical Condition',
    y='Age',
    color='Medical Condition'
).properties(
    title='Promedio de Edad por Condición Médica'
)

# Mostrar el gráfico
chart.display()



/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [19]:
#Hemos finalizado nuestro Dataframe.

df_1.to_csv('df_1.csv', index=False)
from google.colab import files
files.download('df_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>